In [2]:
"""
File v735_optima_VOT.py

Michel Bierlaire
Mon Aug 31 16:16:23 2020

"""
import sys
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.results as res
from biogeme.expressions import Beta, Derive

# Read the data
df = pd.read_csv('optima.dat', '\t')
database = db.Database('boeing', df)

# The following statement allows you to use the names of the
# variable as Python variable.
globals().update(database.variables)

# Define new variables
TimePT_scaled = TimePT / 200
TimeCar_scaled = TimeCar / 200
MarginalCostPT_scaled = MarginalCostPT / 10
CostCarCHF_scaled = CostCarCHF / 10
distance_km_scaled = distance_km / 5
male = Gender == 1
female = Gender == 2
unreportedGender = Gender == -1
fulltime = OccupStat == 1
notfulltime = OccupStat != 1

# Removing some observations
exclude = ((Choice == -1) + ((CarAvail == 3) * (Choice == 1))) != 0
database.remove(exclude)

# List of parameters to be estimated
ASC_CAR = Beta('ASC_CAR', 0, None, None, 0)
ASC_SM = Beta('ASC_SM', 0, None, None, 0)
BETA_TIME_FULLTIME = Beta('BETA_TIME_FULLTIME', 0, None, None, 0)
BETA_TIME_OTHER = Beta('BETA_TIME_OTHER', 0, None, None, 0)
BETA_DIST_MALE = Beta('BETA_DIST_MALE', 0, None, None, 0)
BETA_DIST_FEMALE = Beta('BETA_DIST_FEMALE', 0, None, None, 0)
BETA_DIST_UNREPORTED = Beta('BETA_DIST_UNREPORTED', 0, None, None, 0)
BETA_COST = Beta('BETA_COST', 0, None, None, 0)

# Definition of utility functions:
V_PT = BETA_TIME_FULLTIME * TimePT_scaled * fulltime + \
    BETA_TIME_OTHER * TimePT_scaled * notfulltime + \
    BETA_COST * MarginalCostPT_scaled
V_CAR = ASC_CAR + \
    BETA_TIME_FULLTIME * TimeCar_scaled * fulltime + \
    BETA_TIME_OTHER * TimeCar_scaled * notfulltime + \
    BETA_COST * CostCarCHF_scaled
V_SM = ASC_SM + \
    BETA_DIST_MALE * distance_km_scaled * male + \
    BETA_DIST_FEMALE * distance_km_scaled * female + \
    BETA_DIST_UNREPORTED * distance_km_scaled * unreportedGender

# Associate utility functions with the numbering of alternatives
V = {0: V_PT,
     1: V_CAR,
     2: V_SM}

try:
    m = 'v715_optima_model'
    results = res.bioResults(pickleFile=f'{m}.pickle')
except FileNotFoundError:
    print(f'Run first the script {m}.py in order to generate the file '
          '{m}.pickle.')
    sys.exit()

prob_pt = models.logit(V, None, 0)
prob_car = models.logit(V, None, 1)
prob_sm = models.logit(V, None, 2)

VOT_PT = Derive(V_PT, 'TimePT') / Derive(V_PT, 'MarginalCostPT')
VOT_CAR = Derive(V_CAR, 'TimeCar') / Derive(V_CAR, 'CostCarCHF')


simulate = {
    'VOT PT': 60 * VOT_PT,
    'VOT Car': 60 * VOT_CAR,
    'Full time': fulltime,
    'Weight': Weight}

biosim = bio.BIOGEME(database, simulate)
biosim.modelName = 'v735_optima_VOT01'

# Perform the simulation
simresults = biosim.simulate(results.getBetaValues())

# Calculate confidence intervals
b = results.getBetasForSensitivityAnalysis(biosim.freeBetaNames, size=1000)
left, right = biosim.confidenceIntervals(b, 0.9)


# Apply the weights
simresults['Weighted VOT PT'] = simresults['VOT PT'] * simresults['Weight']
left['Weighted VOT PT'] = left['VOT PT'] * left['Weight']
right['Weighted VOT PT'] = right['VOT PT'] * right['Weight']


simresults['Weighted VOT car'] = simresults['VOT Car'] * simresults['Weight']
left['Weighted VOT car'] = left['VOT Car'] * left['Weight']
right['Weighted VOT car'] = right['VOT Car'] * right['Weight']

# Average VOT for the population
avg_VOT_PT = simresults['Weighted VOT PT'].sum() / simresults['Weight'].sum()
avg_VOT_PT_left = left['Weighted VOT PT'].sum() / left['Weight'].sum()
avg_VOT_PT_right = right['Weighted VOT PT'].sum() / right['Weight'].sum()
avg_VOT_Car = simresults['Weighted VOT car'].sum() / simresults['Weight'].sum()
avg_VOT_Car_left = left['Weighted VOT car'].sum() / left['Weight'].sum()
avg_VOT_Car_right = right['Weighted VOT car'].sum() / right['Weight'].sum()

print(f'Average VOT for public transp.: {avg_VOT_PT:.2f} CHF/h '
      f'[{avg_VOT_PT_left:.2f}:{avg_VOT_PT_right:.2f}]')
print(f'Average VOT for car           : {avg_VOT_Car:.2f} CHF/h '
      f'[{avg_VOT_Car_left:.2f}:{avg_VOT_Car_right:.2f}]')

# Average VOT for full time employees
resFullTime = simresults[simresults['Full time'] == 1]
resFullTime_left = left[simresults['Full time'] == 1]
resFullTime_right = right[simresults['Full time'] == 1]

avg_VOT_PT_full = \
    resFullTime['Weighted VOT PT'].sum() / resFullTime['Weight'].sum()
avg_VOT_PT_full_left = \
    resFullTime_left['Weighted VOT PT'].sum() / resFullTime_left['Weight'].sum()
avg_VOT_PT_full_right = \
    resFullTime_right['Weighted VOT PT'].sum() / resFullTime_right['Weight'].sum()

avg_VOT_Car_full = \
    resFullTime['Weighted VOT car'].sum() / resFullTime['Weight'].sum()
avg_VOT_Car_full_left = \
    resFullTime_left['Weighted VOT car'].sum() / resFullTime_left['Weight'].sum()
avg_VOT_Car_full_right = \
    resFullTime_right['Weighted VOT car'].sum() / resFullTime_right['Weight'].sum()

print(f'Average VOT for public transp. (full time): '
      f'{avg_VOT_PT_full:.2f} CHF/h '
      f'[{avg_VOT_PT_full_left:.2f}:{avg_VOT_PT_full_right:.2f}]')
print(f'Average VOT for car (full time)           : '
      f'{avg_VOT_Car_full:.2f} CHF/h '
      f'[{avg_VOT_Car_full_left:.2f}:{avg_VOT_Car_full_right:.2f}]')

# Average VOT for part time employees
resPartTime = simresults[simresults['Full time'] != 1]
resPartTime_left = left[simresults['Full time'] != 1]
resPartTime_right = right[simresults['Full time'] != 1]

avg_VOT_PT_part = \
    resPartTime['Weighted VOT PT'].sum() / resPartTime['Weight'].sum()
avg_VOT_PT_part_left = \
    resPartTime_left['Weighted VOT PT'].sum() / resPartTime_left['Weight'].sum()
avg_VOT_PT_part_right = \
    resPartTime_right['Weighted VOT PT'].sum() / resPartTime_right['Weight'].sum()

avg_VOT_Car_part = \
    resPartTime['Weighted VOT car'].sum() / resPartTime['Weight'].sum()
avg_VOT_Car_part_left = \
    resPartTime_left['Weighted VOT car'].sum() / resPartTime_left['Weight'].sum()
avg_VOT_Car_part_right = \
    resPartTime_right['Weighted VOT car'].sum() / resPartTime_right['Weight'].sum()

print(f'Average VOT for public transp. (part time): '
      f'{avg_VOT_PT_part:.2f} CHF/h '
      f'[{avg_VOT_PT_part_left:.2f}:{avg_VOT_PT_part_right:.2f}]')
print(f'Average VOT for car (part time)           : '
      f'{avg_VOT_Car_part:.2f} CHF/h '
      f'[{avg_VOT_Car_part_left:.2f}:{avg_VOT_Car_part_right:.2f}]')


Average VOT for public transp.: 3.59 CHF/h [1.42:6.43]
Average VOT for car           : 3.59 CHF/h [1.42:6.43]
Average VOT for public transp. (full time): 6.37 CHF/h [3.77:10.05]
Average VOT for car (full time)           : 6.37 CHF/h [3.77:10.05]
Average VOT for public transp. (part time): 2.00 CHF/h [0.07:4.36]
Average VOT for car (part time)           : 2.00 CHF/h [0.07:4.36]
